<a href="https://colab.research.google.com/github/rakshansingh12/disease-risk-prediction-ml/blob/main/08_biological_interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd

PROJECT_PATH = "/content/drive/MyDrive/disease-risk-prediction-gene-expression"

X = np.load(os.path.join(PROJECT_PATH, "data/processed", "X_variance_selected.npy"))
y = np.load(os.path.join(PROJECT_PATH, "data/processed", "y_labels.npy"))

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = LogisticRegression(max_iter=5000, solver="liblinear")
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000, solver='liblinear')

In [4]:
coefficients = model.coef_[0]

In [5]:
coefficients.shape

(1000,)

In [6]:
raw_path = os.path.join(
    PROJECT_PATH, "data/raw", "GSE5281_expression_matrix.txt.gz"
)

raw_data = pd.read_csv(
    raw_path,
    sep="\t",
    compression="gzip",
    comment="!",
    index_col=0
)

gene_names = raw_data.index.values


In [7]:
# Recompute variance on full data
X_full = raw_data.T
X_log = np.log2(X_full + 1)
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X_log)

gene_variance = np.var(X_scaled, axis=0)

top_gene_indices = np.argsort(gene_variance)[-1000:]
selected_gene_names = gene_names[top_gene_indices]

In [8]:
importance_df = pd.DataFrame({
    "Gene": selected_gene_names,
    "Coefficient": coefficients,
    "AbsCoefficient": np.abs(coefficients)
})

importance_df = importance_df.sort_values(
    by="AbsCoefficient", ascending=False
)

importance_df.head(10)

,Gene,Coefficient,AbsCoefficient
821,226810_at,-0.180359,0.180359
917,1554332_a_at,-0.146745,0.146745
788,212374_at,-0.138732,0.138732
646,231324_at,-0.136692,0.136692
228,215610_at,-0.130171,0.130171
579,209004_s_at,0.127323,0.127323
118,226144_at,0.120406,0.120406
748,203172_at,0.120259,0.120259
662,231175_at,-0.118248,0.118248
525,212017_at,0.117036,0.117036


In [9]:
alz_genes = importance_df[importance_df["Coefficient"] > 0].head(10)
control_genes = importance_df[importance_df["Coefficient"] < 0].head(10)

alz_genes, control_genes

(             Gene  Coefficient  AbsCoefficient
 579   209004_s_at     0.127323        0.127323
 118     226144_at     0.120406        0.120406
 748     203172_at     0.120259        0.120259
 525     212017_at     0.117036        0.117036
 57      243814_at     0.112538        0.112538
 526     204651_at     0.110504        0.110504
 865    1555666_at     0.108752        0.108752
 426    1560474_at     0.108434        0.108434
 266     219562_at     0.106335        0.106335
 898  1553652_a_at     0.102817        0.102817,
              Gene  Coefficient  AbsCoefficient
 821     226810_at    -0.180359        0.180359
 917  1554332_a_at    -0.146745        0.146745
 788     212374_at    -0.138732        0.138732
 646     231324_at    -0.136692        0.136692
 228     215610_at    -0.130171        0.130171
 662     231175_at    -0.118248        0.118248
 749   228269_x_at    -0.114391        0.114391
 610     208475_at    -0.112362        0.112362
 11     1556931_at    -0.110704        

In [16]:
import pandas as pd
import os

PROJECT_PATH = "/content/drive/MyDrive/disease-risk-prediction-gene-expression"

annotation_path = os.path.join(
    PROJECT_PATH, "data/raw", "GPL570_annotation.txt"
)

annotation_df = pd.read_csv(
    annotation_path,
    sep="\t",
    comment="#",
    low_memory=False
)

In [17]:
annotation_df.head()
annotation_df.columns

Index(['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date',
       'Sequence Type', 'Sequence Source', 'Target Description',
       'Representative Public ID', 'Gene Title', 'Gene Symbol',
       'ENTREZ_GENE_ID', 'RefSeq Transcript ID',
       'Gene Ontology Biological Process', 'Gene Ontology Cellular Component',
       'Gene Ontology Molecular Function'],
      dtype='object')

In [18]:
probe_to_gene = annotation_df[["ID", "Gene Symbol"]]
probe_to_gene.head()

,ID,Gene Symbol
0,1007_s_at,DDR1 /// MIR4640
1,1053_at,RFC2
2,117_at,HSPA6
3,121_at,PAX8
4,1255_g_at,GUCA1A


In [19]:
probe_to_gene.columns = ["Probe", "Gene"]

In [20]:
importance_with_genes = importance_df.merge(
    probe_to_gene,
    left_on="Gene",
    right_on="Probe",
    how="left"
)

In [21]:
importance_with_genes.head(10)

,Gene_x,Coefficient,AbsCoefficient,Probe,Gene_y
0,226810_at,-0.180359,0.180359,226810_at,OGFRL1
1,1554332_a_at,-0.146745,0.146745,1554332_a_at,SLCO4A1-AS1
2,212374_at,-0.138732,0.138732,212374_at,FEM1B
3,231324_at,-0.136692,0.136692,231324_at,NaN
4,215610_at,-0.130171,0.130171,215610_at,NaN
5,209004_s_at,0.127323,0.127323,209004_s_at,FBXL5
6,226144_at,0.120406,0.120406,226144_at,MIR1909 /// REXO1
7,203172_at,0.120259,0.120259,203172_at,FXR2
8,231175_at,-0.118248,0.118248,231175_at,BEND6
9,212017_at,0.117036,0.117036,212017_at,FAM168B


In [22]:
importance_with_genes = importance_with_genes[
    ["Probe", "Gene_y", "Coefficient", "AbsCoefficient"]
]

importance_with_genes.columns = [
    "Probe",
    "GeneSymbol",
    "Coefficient",
    "AbsCoefficient"
]

In [23]:
importance_with_genes = importance_with_genes.sort_values(
    by="AbsCoefficient", ascending=False
)

importance_with_genes.head(10)

,Probe,GeneSymbol,Coefficient,AbsCoefficient
0,226810_at,OGFRL1,-0.180359,0.180359
1,1554332_a_at,SLCO4A1-AS1,-0.146745,0.146745
2,212374_at,FEM1B,-0.138732,0.138732
3,231324_at,NaN,-0.136692,0.136692
4,215610_at,NaN,-0.130171,0.130171
5,209004_s_at,FBXL5,0.127323,0.127323
6,226144_at,MIR1909 /// REXO1,0.120406,0.120406
7,203172_at,FXR2,0.120259,0.120259
8,231175_at,BEND6,-0.118248,0.118248
9,212017_at,FAM168B,0.117036,0.117036


In [24]:
importance_with_genes.dropna(subset=["GeneSymbol"]).head(10)

,Probe,GeneSymbol,Coefficient,AbsCoefficient
0,226810_at,OGFRL1,-0.180359,0.180359
1,1554332_a_at,SLCO4A1-AS1,-0.146745,0.146745
2,212374_at,FEM1B,-0.138732,0.138732
5,209004_s_at,FBXL5,0.127323,0.127323
6,226144_at,MIR1909 /// REXO1,0.120406,0.120406
7,203172_at,FXR2,0.120259,0.120259
8,231175_at,BEND6,-0.118248,0.118248
9,212017_at,FAM168B,0.117036,0.117036
10,228269_x_at,KCNIP3,-0.114391,0.114391
11,243814_at,LOC101927377,0.112538,0.112538


In [25]:
alz_genes = importance_with_genes[
    importance_with_genes["Coefficient"] > 0
].dropna(subset=["GeneSymbol"]).head(10)

control_genes = importance_with_genes[
    importance_with_genes["Coefficient"] < 0
].dropna(subset=["GeneSymbol"]).head(10)

alz_genes, control_genes

(           Probe         GeneSymbol  Coefficient  AbsCoefficient
 5    209004_s_at              FBXL5     0.127323        0.127323
 6      226144_at  MIR1909 /// REXO1     0.120406        0.120406
 7      203172_at               FXR2     0.120259        0.120259
 9      212017_at            FAM168B     0.117036        0.117036
 11     243814_at       LOC101927377     0.112538        0.112538
 14     204651_at               NRF1     0.110504        0.110504
 16    1555666_at              PTPRS     0.108752        0.108752
 20     219562_at              RAB26     0.106335        0.106335
 24  1553652_a_at           C18orf54     0.102817        0.102817
 26     223092_at               ANKH     0.102060        0.102060,
            Probe                              GeneSymbol  Coefficient  \
 0      226810_at                                  OGFRL1    -0.180359   
 1   1554332_a_at                             SLCO4A1-AS1    -0.146745   
 2      212374_at                                  

In [26]:
importance_with_genes["GeneSymbol"] = (
    importance_with_genes["GeneSymbol"]
    .str.split("///")
    .str[0]
)

In [27]:
top_genes = importance_with_genes.dropna(
    subset=["GeneSymbol"]
).head(10)

top_genes[["GeneSymbol", "Coefficient"]]

,GeneSymbol,Coefficient
0,OGFRL1,-0.180359
1,SLCO4A1-AS1,-0.146745
2,FEM1B,-0.138732
5,FBXL5,0.127323
6,MIR1909,0.120406
7,FXR2,0.120259
8,BEND6,-0.118248
9,FAM168B,0.117036
10,KCNIP3,-0.114391
11,LOC101927377,0.112538


In [28]:
importance_with_genes.sort_values(by="AbsCoefficient", ascending=False)

,Probe,GeneSymbol,Coefficient,AbsCoefficient
0,226810_at,OGFRL1,-0.180359,0.180359
1,1554332_a_at,SLCO4A1-AS1,-0.146745,0.146745
2,212374_at,FEM1B,-0.138732,0.138732
3,231324_at,NaN,-0.136692,0.136692
4,215610_at,NaN,-0.130171,0.130171
...,...,...,...,...
995,1553831_at,APOBEC3D,0.000175,0.000175
996,203794_at,CDC42BPA,0.000171,0.000171
997,224775_at,IWS1,0.000127,0.000127
998,203164_at,SLC33A1,-0.000090,0.000090


In [29]:
import joblib
import os

PROJECT_PATH = "/content/drive/MyDrive/disease-risk-prediction-gene-expression"

model_path = os.path.join(PROJECT_PATH, "model_logistic.pkl")

joblib.dump(model, model_path)

['/content/drive/MyDrive/disease-risk-prediction-gene-expression/model_logistic.pkl']